<a href="https://colab.research.google.com/github/boysbytes/colab-chatbot/blob/main/colab_api_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Own Chatbot in Google Colab

## Install libraries

In [ ]:
!pip3 install gradio
!pip install -q -U google-generativeai

## Get and configure Gemini API key

1. Open [Google AI Studio](https://aistudio.google.com/app/apikey).
2. Select **Create API Key**. You might need to sign in with your Google account.
3. Copy the API key.

> **Note** \
> Use your API keys securely. Do not share them or embed them in code the public can view.

In [ ]:
# @title Set up API key
gemini_api_key = "" # @param {"type":"string","placeholder":"Enter your Gemini API key."}
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)

# --- Simplified API Key Verification ---
try:
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content("test") # Minimal test prompt
    print("API call successful (preliminary)")
except Exception as e:
    print(f"API call failed: {e}")
    print(f"Error details: {e}")

## Import libraries

In [ ]:
import google.generativeai as genai
import gradio as gr
import re

## Define prompt template and variables

In [ ]:
# Prompt Template
prompt_template = """You are a helpful and friendly mentor.
You will answer user questions in a concise and informative way.
Your responses must be appropriate and relevant to 15 year old students.

Conversation History:
{chat_history_formatted}

User's Message: {user_message}

Chatbot's Response:
"""

# Variables
model_name = "gemini-1.5-flash"
temperature = 0.7

## Create chatbot function

In [ ]:
# Initialize chat history variable
chat_history = [("Chatbot", "Hello! I'm a friendly mentor. How can I help you today?")]

In [ ]:
def contains_prohibited_content(text):
    """Checks if the response contains inappropriate words."""
    prohibited_words = ["kill", "suicide", "sex", "drugs", "violence"]  # Add more as needed
    return any(word in text.lower() for word in prohibited_words)

def generate_gemini_response(user_input, history=[]):
    # Access the global chat_history variable
    global chat_history

    try:
        # Define safety settings
        safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]

        # Include chat history in the prompt
        chat_history_formatted = "\n".join([f"User: {user}\nChatbot: {bot}" for user, bot in chat_history])
        prompt_text = prompt_template.format(chat_history_formatted=chat_history_formatted, user_message=user_input)

        # Format the prompt
        #prompt_text = prompt_template.format(user_message=user_input)

        # Select the Gemini model with safety settings
        model = genai.GenerativeModel(model_name, safety_settings=safety_settings)

        # Generate content with temperature
        response = model.generate_content(
            prompt_text,
            generation_config=genai.types.GenerationConfig(temperature=temperature)
        )
        chatbot_response = response.text


        # Moderate content manually
        if contains_prohibited_content(chatbot_response):
            return "Sorry, I can't provide an answer to that."

        # Update chat history
        chat_history.append((user_input, chatbot_response))

        return chatbot_response

    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        return "Sorry, something went wrong."


## Create the UI

In [ ]:
iface = gr.ChatInterface(
    fn=generate_gemini_response,
    title="Gemini Chatbot",
    description="Chat with Gemini",
    chatbot=gr.Chatbot(height=300, value=chat_history) # Optional: Adjust chatbot display height
)
iface.launch(share=True, debug=True)